In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

with open('/s3/misha/data_dir/PMC_patients/PMC-Patients-V2.json', 'r') as f:
    df_patients_v2 = json.load(f)

len(df_patients_v2)

250294

In [14]:
import requests

def get_full_text(id):
    response = requests.get(f'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{id}/unicode')

    if 'No result can be found' in response.text:
        return "no text"
    else:
        json_file = response.json()
        full_text_pieces = []
        id = json_file[0]['documents'][0]['id']
        passages = json_file[0]['documents'][0]['passages']

        for passage in passages:
            full_text_pieces.append(passage['text'])
        return " ".join(full_text_pieces)


full_text = get_full_text('16923179')

In [15]:
len(full_text)

13418

In [ ]:
from tqdm import tqdm
import pandas as pd
import pyarrow.parquet as pq


save_path = '/s3/misha/data_dir/PMC_patients/full_texts_PMC-Patients-V2.parquet'
records = []
for patient in tqdm(df_patients_v2):
    pmid = patient['PMID']
    patient_uid = patient['patient_uid']
    full_text = get_full_text(pmid)
    
    record = {
        'PMID': pmid,
        'patient_uid': patient_uid,
        'title': patient['title'],
        'full_text': full_text
    }
    records.append(record)

df = pd.DataFrame(records)
df.to_parquet('/s3/misha/data_dir/PMC_patients/full_texts_PMC-Patients-V2.parquet', engine='pyarrow', compression='snappy')

In [19]:
df[df['full_text'] == 'no text']

,PMID,patient_uid,title,full_text


In [1]:
import pandas as pd

df = pd.read_parquet('/s3/misha/data_dir/PMC_patients/full_texts_PMC-Patients-V2.parquet', engine='pyarrow')

In [4]:
df[df['full_text'] == 'no text']

,PMID,patient_uid,title,full_text
3347,11799754,2730271-1,First Case of Human Rabies in Chile Caused by ...,no text
3348,11799760,2730276-1,Nosocomial Endocarditis Caused by Corynebacter...,no text
3349,11799760,2730276-2,Nosocomial Endocarditis Caused by Corynebacter...,no text
3350,11799760,2730276-3,Nosocomial Endocarditis Caused by Corynebacter...,no text
3351,11799757,2730277-1,Human Herpesvirus 6-Associated Hemophagocytic ...,no text
...,...,...,...,...
158239,21120194,2983030-1,Development of acute myocardial infarction in ...,no text
158240,21120212,2983040-1,Long-term survival in a patient with acute pro...,no text
158241,21120211,2983047-1,Acquired factor VIII deficiency after consumin...,no text
158242,21103106,2984327-1,The use of the ice pack test in myasthenia gravis,no text
